In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import clear_output, HTML, Image
import time
import warnings
warnings.filterwarnings('ignore') 

%config InlineBackend.figure_format = 'svg'
plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "stix",
    "font.serif": ["Times New Roman"],
    "font.size": 12
})

In [ ]:
import sys
sys.path.append('..')
from latticegas import LatticeGas

In [ ]:
# Вариант 1 (N = 5)
N = 5

# Размеры расчётной области
n_x = 420
n_y = 180

# Параметры симуляции
n_step = 1000
step_frame = 1

# Параметры цилиндра
xc = int(n_x / 4)  # 105
yc = int(n_y / 2)  # 90
r = int(n_y / 9)   # 20

# Диапазоны параметров
u_lb_min = 0.01
u_lb_max = 0.1
Re_min = 20
Re_max = 1000

# Расчёт конкретных значений
u_lb = u_lb_min + 2 * (u_lb_max - u_lb_min) / (N - 1)
Re = int(Re_min + 2 * (Re_max - Re_min) / (N - 1))

# Вывод параметров
print("=" * 60)
print("ПАРАМЕТРЫ СИМУЛЯЦИИ")
print("=" * 60)
print(f"Вариант: N = {N}")
print(f"Размер сетки: {n_x} × {n_y}")
print(f"Цилиндр: центр ({xc}, {yc}), радиус {r}")
print(f"Скорость на входе (u_lb): {u_lb:.4f}")
print(f"Число Рейнольдса (Re): {Re}")
print(f"Количество шагов: {n_step}")
print(f"Сохранение полей: каждый {step_frame}-й шаг")
print("=" * 60)

In [ ]:
parametrs = {
    'nx': n_x,
    'ny': n_y, 
    'u_lb': u_lb,
    'Re': Re
}

obstacle = {
    'xc': xc,
    'yc': yc,
    'r': r
}

model = LatticeGas(parametrs, obstacle)

print("\nЗапуск симуляции LBM...")
start_time = time.time()

model.solve(n_step=n_step, step_frame=step_frame)

elapsed_time = time.time() - start_time
print(f"\nВремя выполнения симуляции: {elapsed_time:.2f} секунд")
print(f"Среднее время на шаг: {elapsed_time/n_step*1000:.2f} мс")

In [ ]:
density = np.array(model.field_den)
velocity = np.array(model.field_u)
pressure = np.array(model.field_p)

np.save('density_zen.npy', density)
np.save('velocity_zen.npy', velocity)
np.save('pressure_zen.npy', pressure)

print("=" * 60)
print("РЕЗУЛЬТАТЫ СОХРАНЕНЫ")
print("=" * 60)
print(f"Поле плотности: density_zen.npy ({density.shape})")
print(f"Поле скорости:  velocity_zen.npy ({velocity.shape})")
print(f"Поле давления:  pressure_zen.npy ({pressure.shape})")
print("=" * 60)

In [ ]:
obstacle_mask = np.zeros((model.nx, model.ny), dtype=bool)
obstacle_mask[model.obstacle['x'], model.obstacle['y']] = True
obst = obstacle_mask.T 

print(f"Маска создана. Ячеек в препятствии: {np.sum(obstacle_mask)}")

In [ ]:
print("Запуск анимации в реальном времени...")
print("Для остановки: Kernel → Interrupt")

for j in range(0, min(n_step, len(velocity)), 50):
    i = j
    
    clear_output(wait=True)
    
    fig, ax = plt.subplots(figsize=(8, 5), dpi=150)
    divider = make_axes_locatable(ax)
    
    velocity_frame = velocity[i].T.copy()
    velocity_frame[obst] = np.nan
    
    im = ax.imshow(velocity_frame, cmap="jet", vmin=0, vmax=0.15)
    ax.invert_yaxis()
    ax.set_title(f'Итерация {(i+1)*step_frame}', fontsize=17, pad=10)
    ax.set_xlabel(r'$x$', fontsize=16)
    ax.set_ylabel(r'$y$', fontsize=16)
    ax.set_facecolor("lightgray")
    
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cb = fig.colorbar(im, cax=cax, orientation='vertical')
    cb.set_label(r"Скорость", fontsize=16)
    
    plt.show()
    
    if i == min(n_step, len(velocity)) - 50:
        fig.savefig(f"velocity_final.jpg", dpi=150, bbox_inches='tight')
        print(f"\nФинальный кадр сохранён: velocity_final.jpg")
    
    time.sleep(0.3) 

print("Анимация завершена.")

In [ ]:
import matplotlib.animation as animation

fig, ax = plt.subplots(figsize=(9, 5))
vel_frame = velocity[0].T.copy()
vel_frame[obst] = np.nan
img = ax.imshow(vel_frame, cmap='jet', vmin=0, vmax=0.15, animated=True)
ax.set_title('Обтекание цилиндра')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(img, label='Скорость')

def update(i):
    vel_frame = velocity[i*5].T.copy() 
    vel_frame[obst] = np.nan
    img.set_array(vel_frame)
    ax.set_title(f'Шаг {i*5}')
    return img,

ani = animation.FuncAnimation(fig, update, frames=range(0, len(velocity)//5, 2), interval=150)
ani.save('lattice_gas_animation.gif', writer='pillow', fps=5, dpi=100)
plt.close(fig)

print("GIF анимация создана: lattice_gas_animation.gif")
Image('lattice_gas_animation.gif')

In [ ]:
print("=" * 60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ")
print("=" * 60)

max_velocities = [np.nanmax(v) for v in velocity]

plt.figure(figsize=(10, 4))
plt.plot(max_velocities, linewidth=2)
plt.xlabel('Шаг по времени', fontsize=12)
plt.ylabel('Максимальная скорость', fontsize=12)
plt.title('Эволюция максимальной скорости', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('max_velocity_evolution.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ График эволюции скорости сохранён")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

vel_final = velocity[-1].T.copy()
vel_final[obst] = np.nan
im1 = axes[0].imshow(vel_final, cmap='jet', vmin=0, vmax=0.15)
axes[0].set_title('Финальное поле скорости', fontsize=13)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
plt.colorbar(im1, ax=axes[0])

den_final = density[-1].T.copy()
den_final[obst] = np.nan
im2 = axes[1].imshow(den_final, cmap='viridis')
axes[1].set_title('Финальное поле плотности', fontsize=13)
axes[1].set_xlabel('X')
plt.colorbar(im2, ax=axes[1])

pres_final = pressure[-1].T.copy()
pres_final[obst] = np.nan
im3 = axes[2].imshow(pres_final, cmap='plasma')
axes[2].set_title('Финальное поле давления', fontsize=13)
axes[2].set_xlabel('X')
plt.colorbar(im3, ax=axes[2])

plt.suptitle(f'Финальные поля (Re={Re}, u_lb={u_lb:.3f})', fontsize=15)
plt.tight_layout()
plt.savefig('final_fields.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Финальные поля сохранены")

print("=" * 60)
print("ВИЗУАЛИЗАЦИЯ ЗАВЕРШЕНА")
print("=" * 60)

In [ ]:
import os

print("=" * 60)
print("СОЗДАННЫЕ ФАЙЛЫ")
print("=" * 60)

files = [
    ('density_zen.npy', 'Поле плотности (бинарный)'),
    ('velocity_zen.npy', 'Поле скорости (бинарный)'),
    ('pressure_zen.npy', 'Поле давления (бинарный)'),
    ('velocity_final.jpg', 'Финальный кадр скорости'),
    ('lattice_gas_animation.gif', 'GIF анимация'),
    ('max_velocity_evolution.png', 'График эволюции скорости'),
    ('final_fields.png', 'Финальные поля')
]

for filename, description in files:
    if os.path.exists(filename):
        size = os.path.getsize(filename)
        size_str = f"{size/1024:.1f} KB" if size < 1024*1024 else f"{size/(1024*1024):.1f} MB"
        print(f"✓ {filename:30} {description:35} ({size_str})")
    else:
        print(f"✗ {filename:30} {description:35} (не найден)")

print("\n" + "=" * 60)
print("ВЫВОДЫ")
print("=" * 60)
print("1. Модель успешно рассчитала обтекание цилиндра при Re =", Re)
print("2. Визуализация показывает формирование вихревой дорожки")
print("3. Максимальная скорость установилась на уровне", f"{max(max_velocities):.3f}")
print("4. Все файлы сохранены и готовы для включения в отчёт")